In [44]:
import numpy as np
from geopy.distance import geodesic
import random
from itertools import combinations
from tqdm import tqdm
import pandas as pd



In [45]:
# Define the TSP cost calculation
def calculate_route_distance(route):
    """Calculates the total distance of the TSP route."""
    return sum(DIST_MATRIX[route[i], route[i+1]] for i in range(len(route) - 1)) + DIST_MATRIX[route[-1], route[0]]

# Genetic Algorithm functions
def fitness(route):
    """Calculates the fitness (inverse of distance) of a route."""
    distance = calculate_route_distance(route)
    return 1 / distance if distance > 0 else float('inf')

def initial_population(pop_size):
    """Generates an initial population of random routes."""
    population = []
    for _ in range(pop_size):
        route = list(range(len(CITIES)))
        random.shuffle(route)
        population.append(route)
    return population

def selection(population, fitnesses):
    """Selects two parents using weighted random choice based on fitness."""
    selected = random.choices(population, weights=fitnesses, k=2)
    return selected[0], selected[1]

def crossover(parent1, parent2):
    """Performs ordered crossover between two parents to produce a child."""
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [-1] * len(parent1)
    child[start:end] = parent1[start:end]
    pointer = end
    for city in parent2:
        if city not in child:
            if pointer >= len(parent1):
                pointer = 0
            child[pointer] = city
            pointer += 1
    return child

def mutate(route, mutation_rate):
    """Mutates a route by swapping two cities with a given probability."""
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]


In [46]:
# Genetic Algorithm Parameters
POP_SIZE = 500
GENS = 500
MUTATION_RATE = 0.05
ELITISM = 0.05

def genetic_algorithm():
    """Solves the TSP using a Genetic Algorithm and returns the best route found along with the number of steps."""
    population = initial_population(POP_SIZE)
    steps = 0

    for gen in tqdm(range(GENS)):
        steps += 1

        # Evaluate fitness for each route
        fitnesses = [fitness(route) for route in population]

        # Elitism: carry over the top-performing routes to the new population
        elite_count = int(ELITISM * POP_SIZE)
        sorted_population = [route for _, route in sorted(zip(fitnesses, population), reverse=True)]
        new_population = sorted_population[:elite_count]

        while len(new_population) < POP_SIZE:
            parent1, parent2 = selection(population, fitnesses)
            child = crossover(parent1, parent2)
            mutate(child, MUTATION_RATE)
            new_population.append(child)

        population = new_population

    best_route = min(population, key=calculate_route_distance)

    if best_route[0] != best_route[-1]:
        best_route.append(best_route[0])

    return best_route, calculate_route_distance(best_route), steps

# Steady-State Genetic Algorithm (Slower but potentially more accurate)
def steady_state_genetic_algorithm():
    """Solves the TSP using a Steady-State Genetic Algorithm and returns the best route found along with the number of steps."""
    population = initial_population(POP_SIZE)
    best_route = None
    best_distance = float('inf')
    steps = 0  # Initialize step counter

    for gen in tqdm(range(GENS)):
        steps += 1  # Increment steps for each generation

        fitnesses = [fitness(route) for route in population]

        # Select two parents and generate a child
        parent1, parent2 = selection(population, fitnesses)
        child = crossover(parent1, parent2)
        mutate(child, MUTATION_RATE)

        child_distance = calculate_route_distance(child)
        worst_idx = fitnesses.index(min(fitnesses))

        if child_distance < calculate_route_distance(population[worst_idx]):
            population[worst_idx] = child

        # Track the best solution found
        if child_distance < best_distance:
            best_route = child[:]
            best_distance = child_distance

    # Ensure the route is a closed loop
    if best_route[0] != best_route[-1]:
        best_route.append(best_route[0])

    return best_route, best_distance, steps


In [49]:
# Load city data with error handling
try:
    file_path = 'path/to/your/china.csv'  # Replace with the actual path to your CSV file
    CITIES = pd.read_csv('/content/china.csv', header=None, names=['name', 'lat', 'lon'])
except FileNotFoundError:
    print("Error: File not found. Please check the file path.")
    raise

DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

# Execute and Compare Both Algorithms
# Approximate Solution (Genetic Algorithm)
best_approx_route, approx_distance, approx_steps = genetic_algorithm()
print("\nApproximate Solution (Genetic Algorithm):")
print(f"Best Route: {best_approx_route}")
print(f"Total Distance: {approx_distance:.2f} km")
print(f"Number of Steps (Generations): {approx_steps}")

#(Steady-State Genetic Algorithm)
best_steady_route, steady_distance, steady_steps = steady_state_genetic_algorithm()
print("\nSlower but More Accurate Solution (Steady-State Genetic Algorithm):")
print(f"Best Route: {best_steady_route}")
print(f"Total Distance: {steady_distance:.2f} km")
print(f"Number of Steps (Generations): {steady_steps}")

100%|██████████| 500/500 [20:53<00:00,  2.51s/it]



Approximate Solution (Genetic Algorithm):
Best Route: [270, 490, 302, 242, 511, 149, 607, 304, 195, 255, 81, 111, 250, 136, 109, 297, 51, 330, 161, 301, 135, 347, 113, 574, 663, 712, 562, 502, 7, 403, 623, 234, 538, 240, 132, 194, 83, 268, 238, 477, 252, 45, 633, 641, 602, 227, 487, 386, 680, 375, 209, 80, 137, 152, 25, 138, 606, 601, 5, 414, 397, 239, 369, 286, 38, 87, 33, 125, 416, 282, 589, 632, 65, 542, 661, 649, 722, 655, 281, 581, 478, 271, 455, 259, 647, 458, 535, 569, 98, 406, 390, 559, 499, 383, 431, 419, 451, 636, 507, 434, 232, 377, 319, 290, 197, 714, 392, 257, 440, 323, 540, 545, 686, 381, 407, 690, 275, 402, 84, 317, 253, 140, 640, 657, 389, 251, 130, 41, 182, 307, 326, 667, 470, 492, 344, 133, 384, 277, 580, 621, 583, 526, 410, 205, 682, 43, 39, 165, 201, 482, 101, 13, 352, 77, 524, 415, 380, 178, 658, 417, 515, 241, 121, 120, 556, 552, 134, 49, 16, 318, 462, 96, 349, 662, 354, 9, 168, 200, 446, 236, 215, 308, 679, 675, 208, 505, 18, 644, 444, 627, 248, 591, 366, 6, 710

100%|██████████| 500/500 [01:35<00:00,  5.25it/s]


Slower but More Accurate Solution (Steady-State Genetic Algorithm):
Best Route: [423, 680, 408, 32, 364, 711, 159, 409, 206, 71, 317, 10, 299, 20, 81, 234, 400, 247, 550, 421, 363, 672, 308, 104, 486, 526, 35, 76, 66, 496, 350, 22, 291, 57, 318, 91, 362, 537, 171, 17, 471, 586, 259, 195, 28, 280, 284, 27, 688, 563, 53, 179, 524, 177, 709, 105, 445, 682, 72, 434, 724, 184, 596, 698, 460, 185, 412, 456, 613, 448, 94, 125, 465, 96, 520, 131, 700, 302, 160, 535, 647, 352, 677, 540, 300, 199, 482, 620, 477, 366, 145, 378, 619, 163, 422, 419, 227, 343, 243, 210, 155, 640, 696, 427, 658, 499, 659, 226, 132, 293, 113, 161, 146, 245, 491, 14, 167, 610, 599, 446, 652, 587, 548, 332, 444, 722, 615, 111, 513, 251, 0, 169, 576, 43, 525, 183, 137, 134, 669, 689, 271, 714, 151, 114, 207, 84, 635, 248, 565, 49, 512, 23, 723, 416, 541, 716, 250, 636, 340, 178, 418, 371, 261, 309, 509, 208, 591, 219, 239, 458, 140, 242, 112, 26, 414, 339, 452, 687, 674, 498, 135, 390, 396, 34, 75, 122, 474, 152, 694, 1

In [48]:
# Load city data with error handling
try:
    file_path = 'path/to/your/italy.csv'  # Replace with the actual path to your CSV file
    CITIES = pd.read_csv('/content/italy.csv', header=None, names=['name', 'lat', 'lon'])
except FileNotFoundError:
    print("Error: File not found. Please check the file path.")
    raise

# Generate distance matrix
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

# Execute and Compare Both Algorithms
# Approximate Solution (Genetic Algorithm)
best_approx_route, approx_distance, approx_steps = genetic_algorithm()
print("\nApproximate Solution (Genetic Algorithm):")
print(f"Best Route: {best_approx_route}")
print(f"Total Distance: {approx_distance:.2f} km")
print(f"Number of Steps (Generations): {approx_steps}")

#(Steady-State Genetic Algorithm)
best_steady_route, steady_distance, steady_steps = steady_state_genetic_algorithm()
print("\nSlower but More Accurate Solution (Steady-State Genetic Algorithm):")
print(f"Best Route: {best_steady_route}")
print(f"Total Distance: {steady_distance:.2f} km")
print(f"Number of Steps (Generations): {steady_steps}")

100%|██████████| 500/500 [00:22<00:00, 22.20it/s]



Approximate Solution (Genetic Algorithm):
Best Route: [20, 18, 22, 42, 13, 16, 29, 26, 39, 15, 14, 21, 35, 38, 2, 11, 1, 27, 34, 24, 8, 37, 31, 17, 7, 36, 10, 0, 33, 30, 12, 4, 19, 25, 32, 6, 44, 40, 5, 41, 43, 23, 45, 9, 28, 3, 20]
Total Distance: 5303.53 km
Number of Steps (Generations): 500


100%|██████████| 500/500 [00:05<00:00, 94.29it/s] 


Slower but More Accurate Solution (Steady-State Genetic Algorithm):
Best Route: [35, 2, 1, 36, 45, 3, 43, 32, 38, 22, 24, 39, 30, 15, 14, 17, 37, 8, 4, 11, 21, 26, 33, 27, 31, 5, 41, 28, 0, 29, 13, 7, 42, 12, 20, 18, 23, 10, 40, 16, 25, 9, 6, 44, 19, 34, 35]
Total Distance: 14906.86 km
Number of Steps (Generations): 500
